In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# ===============================
file_path = "C:/Users/Jia/Desktop/Final_data.xlsx"
df = pd.read_excel(file_path)
# print(df.head(), df.columns.tolist())

# # ===============================
target_var = ['CDS_spread']
esg_scores = [
    'ESG Score', 'ESG Combined Score', 'ESG Controversies Score',
    'Environment Pillar Score', 'Social Pillar Score', 'Governance Pillar Score'
]
esg_subscores = [
    'Emissions Score', 'Environmental Innovation Score', 'Community Score',
    'CSR Strategy Score', 'Human Rights Score', 'Product Responsibility Score',
    'Resource Use Score', 'Workforce Score', 'Management Score',
    'Shareholders Score', 'TRDIR Diversity Score', 'TRDIR Inclusion Score',
    'TRDIR People Development Score', 'TRDIR Controversies Score'
]
financial_controls = [
    'LIQUIDITY RATIO', 'LEVERAGE RATIO', 'RETURN ON ASSETS',
    'INTEREST COVERAGE RATIO', 'MARKET TO BOOK RATIO', 'RISK FREE RATE',
    'HISTORICAL VOLATILITY 5 YEARS', 'RETURN ON EQUITY - TOTAL (%)',
    'MARKET VALUE', 'TOTAL ASSETS'
]

selected_columns = target_var + esg_scores + esg_subscores + financial_controls
# df_analysis = df[selected_columns].dropna()
#
# # ===============================
# print("=== Descriptive Statistics ===")
# print(df_analysis.describe())
#
# # ===============================
# print("\n=== Correlation Matrix Head ===")
# selected_columns = target_var + esg_scores + esg_subscores + financial_controls
# df_corr = df[selected_columns].dropna()
#
# # === 定义绘图函数 ===
# def plot_corr_with_target_precise(df, vars_list, title):
#     corr_vals = df[['CDS_spread'] + vars_list].corr().loc[vars_list, 'CDS_spread'].to_frame()
#     corr_vals.columns = ['Correlation']
#     corr_vals = corr_vals.round(4)
#
#     plt.figure(figsize=(8, len(vars_list) * 0.5 + 1))
#     sns.heatmap(corr_vals, annot=True, fmt=".4f", cmap='coolwarm', center=0)
#     plt.title(title)
#     plt.tight_layout()
#     plt.show()
#
#
# # ======
# plot_corr_with_target_precise(df_corr, esg_scores, 'Correlation: CDS Spread vs ESG Summary Scores')
# plot_corr_with_target_precise(df_corr, esg_subscores, 'Correlation: CDS Spread vs ESG Subscores')
# plot_corr_with_target_precise(df_corr, financial_controls, 'Correlation: CDS Spread vs Financial Controls')
#
#
#
# # ===============================
# df.isnull().sum()
# missing_values = df.isnull().sum()[df.isnull().sum() > 0]
# print(missing_values)
# z_scores = np.abs(stats.zscore(df_analysis))
# outlier_counts = (z_scores > 3).sum(axis=0)
# print("\n=== Outlier Count per Variable (Z-score > 3) ===")
# print(pd.DataFrame({'Variable': df_analysis.columns, 'Outlier Count': outlier_counts}).sort_values(by='Outlier Count', ascending=False))


# Lasso regression

# 合并所有用于回归的变量
df = df[selected_columns].dropna()

# 拆分 X 与 y，并标准化
X = df.drop(columns=target_var)
y = df[target_var[0]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# # 拆分训练测试集
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
#
# # 执行 LASSO 回归
# lasso = LassoCV(cv=5, max_iter=10000, random_state=42)
# lasso.fit(X_train, y_train)
#
# # 提取非零系数
# lasso_coef = pd.Series(lasso.coef_, index=X.columns)
# selected_features = lasso_coef[lasso_coef != 0].sort_values(ascending=False)
# # 打印结果
# print("✅ Non-zero LASSO Coefficients:\n")
# print(selected_features)
#
# plt.figure(figsize=(10, 6))
# selected_features.plot(kind='barh')
# plt.title('Selected Features from LASSO Regression')
# plt.xlabel('Coefficient')
# plt.tight_layout()
# plt.show()

# Random forest
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(r2, rmse, mae)

# plt.figure(figsize=(8, 6))
# plt.scatter(y_test, y_pred, alpha=0.5, edgecolors='k')
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
# plt.xlabel("Actual CDS Spread")
# plt.ylabel("Predicted CDS Spread")
# plt.title("Random Forest: Actual vs. Predicted")
# plt.grid(True)
# plt.tight_layout()
# plt.show()
#
# # 注意：如果你用的是 X_scaled，就需要用原始列名来标注
# feature_names = X.columns if isinstance(X, pd.DataFrame) else [f"X{i}" for i in range(X.shape[1])]
# importances = rf.feature_importances_
# forest_importances = pd.Series(importances, index=feature_names).sort_values(ascending=True)
#
# plt.figure(figsize=(10, 8))
# forest_importances.plot(kind='barh')
# plt.title("Random Forest Feature Importances")
# plt.xlabel("Importance (Gini-based)")
# plt.tight_layout()
# plt.show()

# 获取特征名称（确保为 DataFrame 格式）
feature_names = X.columns if isinstance(X, pd.DataFrame) else [f"X{i}" for i in range(X.shape[1])]
importances = rf.feature_importances_

# 构建 Series 并按重要性降序排序
feature_importance_series = pd.Series(importances, index=feature_names).sort_values(ascending=False)

# 打印所有变量及其重要性
print("✅ Random Forest - All Feature Importances:\n")
print(feature_importance_series)

In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# ===============================
file_path = "C:/Users/Jia/Desktop/Final_data.xlsx"
df = pd.read_excel(file_path)
# print(df.head(), df.columns.tolist())

# # ===============================
target_var = ['CDS_spread']
esg_scores = [
    'ESG Score', 'ESG Combined Score', 'ESG Controversies Score',
    'Environment Pillar Score', 'Social Pillar Score', 'Governance Pillar Score'
]
esg_subscores = [
    'Emissions Score', 'Environmental Innovation Score', 'Community Score',
    'CSR Strategy Score', 'Human Rights Score', 'Product Responsibility Score',
    'Resource Use Score', 'Workforce Score', 'Management Score',
    'Shareholders Score', 'TRDIR Diversity Score', 'TRDIR Inclusion Score',
    'TRDIR People Development Score', 'TRDIR Controversies Score'
]
financial_controls = [
    'LIQUIDITY RATIO', 'LEVERAGE RATIO', 'RETURN ON ASSETS',
    'INTEREST COVERAGE RATIO', 'MARKET TO BOOK RATIO', 'RISK FREE RATE',
    'HISTORICAL VOLATILITY 5 YEARS', 'RETURN ON EQUITY - TOTAL (%)',
    'MARKET VALUE', 'TOTAL ASSETS'
]

selected_columns = target_var + esg_scores + esg_subscores + financial_controls
# df_analysis = df[selected_columns].dropna()
#
# # ===============================
# print("=== Descriptive Statistics ===")
# print(df_analysis.describe())
#
# # ===============================
# print("\n=== Correlation Matrix Head ===")
# selected_columns = target_var + esg_scores + esg_subscores + financial_controls
# df_corr = df[selected_columns].dropna()
#
# # === 定义绘图函数 ===
# def plot_corr_with_target_precise(df, vars_list, title):
#     corr_vals = df[['CDS_spread'] + vars_list].corr().loc[vars_list, 'CDS_spread'].to_frame()
#     corr_vals.columns = ['Correlation']
#     corr_vals = corr_vals.round(4)
#
#     plt.figure(figsize=(8, len(vars_list) * 0.5 + 1))
#     sns.heatmap(corr_vals, annot=True, fmt=".4f", cmap='coolwarm', center=0)
#     plt.title(title)
#     plt.tight_layout()
#     plt.show()
#
#
# # ======
# plot_corr_with_target_precise(df_corr, esg_scores, 'Correlation: CDS Spread vs ESG Summary Scores')
# plot_corr_with_target_precise(df_corr, esg_subscores, 'Correlation: CDS Spread vs ESG Subscores')
# plot_corr_with_target_precise(df_corr, financial_controls, 'Correlation: CDS Spread vs Financial Controls')
#
#
#
# # ===============================
# df.isnull().sum()
# missing_values = df.isnull().sum()[df.isnull().sum() > 0]
# print(missing_values)
# z_scores = np.abs(stats.zscore(df_analysis))
# outlier_counts = (z_scores > 3).sum(axis=0)
# print("\n=== Outlier Count per Variable (Z-score > 3) ===")
# print(pd.DataFrame({'Variable': df_analysis.columns, 'Outlier Count': outlier_counts}).sort_values(by='Outlier Count', ascending=False))






# Lasso regression

# 合并所有用于回归的变量
df = df[selected_columns].dropna()

# 拆分 X 与 y，并标准化
X = df.drop(columns=target_var)
y = df[target_var[0]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# # 拆分训练测试集
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
#
# # 执行 LASSO 回归
# lasso = LassoCV(cv=5, max_iter=10000, random_state=42)
# lasso.fit(X_train, y_train)
#
# # 提取非零系数
# lasso_coef = pd.Series(lasso.coef_, index=X.columns)
# selected_features = lasso_coef[lasso_coef != 0].sort_values(ascending=False)
# # 打印结果
# print("✅ Non-zero LASSO Coefficients:\n")
# print(selected_features)
#
# plt.figure(figsize=(10, 6))
# selected_features.plot(kind='barh')
# plt.title('Selected Features from LASSO Regression')
# plt.xlabel('Coefficient')
# plt.tight_layout()
# plt.show()

# Random forest
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(r2, rmse, mae)

# plt.figure(figsize=(8, 6))
# plt.scatter(y_test, y_pred, alpha=0.5, edgecolors='k')
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
# plt.xlabel("Actual CDS Spread")
# plt.ylabel("Predicted CDS Spread")
# plt.title("Random Forest: Actual vs. Predicted")
# plt.grid(True)
# plt.tight_layout()
# plt.show()
#
# # 注意：如果你用的是 X_scaled，就需要用原始列名来标注
# feature_names = X.columns if isinstance(X, pd.DataFrame) else [f"X{i}" for i in range(X.shape[1])]
# importances = rf.feature_importances_
# forest_importances = pd.Series(importances, index=feature_names).sort_values(ascending=True)
#
# plt.figure(figsize=(10, 8))
# forest_importances.plot(kind='barh')
# plt.title("Random Forest Feature Importances")
# plt.xlabel("Importance (Gini-based)")
# plt.tight_layout()
# plt.show()

# 获取特征名称（确保为 DataFrame 格式）
feature_names = X.columns if isinstance(X, pd.DataFrame) else [f"X{i}" for i in range(X.shape[1])]
importances = rf.feature_importances_

# 构建 Series 并按重要性降序排序
feature_importance_series = pd.Series(importances, index=feature_names).sort_values(ascending=False)

# 打印所有变量及其重要性
print("✅ Random Forest - All Feature Importances:\n")
print(feature_importance_series)